In [170]:
# Cell 1: Import libraries and set display options
import pandas as pd
import numpy as np

# Display all columns
pd.set_option("display.max_columns", None)

In [171]:
# Cell 2: Define paths to the CSV files and read them into DataFrames

# Paths to the CSV-files
data_sample_path = "data_sample_AufgDataScience_Case.csv"
event_locations_path = "event_locations_DataScience_Case.csv"

# Read the files into Pandas Dataframes
data_sample = pd.read_csv(data_sample_path, sep=";")
event_locations = pd.read_csv(event_locations_path, sep=";")

In [172]:
# Cell 3: Initial examination of the data

# Show first five rows to check contents of Dataframes
print(data_sample.head())
print(event_locations.head())

  ContainerNumber  ShipmentNumber    PoCreationDate     Material  \
0     HLXU8170197         4868820  19.02.2018 00:00  A2760101414   
1     HLXU8170197         4868820  19.02.2018 00:00  A2760101414   
2     HLXU8170197         4868820  19.02.2018 00:00  A2760101414   
3     HLXU8170197         4868820  19.02.2018 00:00  A2760101414   
4     HLXU8170197         4868820  19.02.2018 00:00  A2760101414   

   ShipmentQuantity Controller        EventName         EventTime  \
0              72.0        T15     Loading Ship  07.04.2018 13:58   
1              72.0        T15   Departure Ship  07.04.2018 19:05   
2              72.0        T15   Unloading Ship  21.04.2018 06:19   
3              72.0        T15  Departure Truck  21.04.2018 11:42   
4              72.0        T15    Arrival Truck  08.05.2018 18:12   

   EventMessageTime             EventLocation   VesselName PortofDischarge  \
0  08.04.2018 03:19  urn:jaif:id:loc:26LNLRTM  NYK ROMULUS             NaN   
1  09.04.2018 16:29 

In [173]:
# Cell 4: Count and drop duplicate entries

# Count and drop duplicate entries
data_sample_duplicates = data_sample.duplicated().sum()
print("Count of double rows: ", data_sample_duplicates)

row_count_original_df = len(data_sample)
print("The original dataframe has", row_count_original_df, "rows.")

data_sample_no_duplicates = data_sample.drop_duplicates(keep=False)
row_count_no_duplicates_df = len(data_sample_no_duplicates)
print("The dataframe without duplicates has", row_count_no_duplicates_df, "rows.")

Count of double rows:  138
The original dataframe has 35306 rows.
The dataframe without duplicates has 35030 rows.


In [174]:
# Cell 5: Analyse missing values

data_sample_count = data_sample_no_duplicates.isnull().sum()
percent_missing = round(((data_sample_count.sum() / np.prod(data_sample_no_duplicates.shape)) * 100), 2)
# Counting and printing the amount of missing values
print("Count of missing values", "\n", data_sample_count)
# Printing the percentage of missing values
print("Percent of missing data:", percent_missing)

Count of missing values 
 ContainerNumber         0
ShipmentNumber          0
PoCreationDate          0
Material                0
ShipmentQuantity       17
Controller              0
EventName               0
EventTime               0
EventMessageTime        0
EventLocation         473
VesselName              0
PortofDischarge     14085
Time2Arrival        14113
dtype: int64
Percent of missing data: 6.3


In [175]:
# Cell 6: Merge DataFrames and clean up columns

# Merge Dataframes on column "EventLocation" and drop column "EventLocation"
merged_data = pd.merge(data_sample_no_duplicates, event_locations, on="EventLocation", how="left")
merged_data = merged_data.drop(columns=["EventLocation"])
# Rename column "Bezeichnung" to "Description"
merged_data.rename(columns={"Bezeichnung": "Description"}, inplace=True)

In [176]:
# Cell 7: Deal with cryptic codes in columns "Description" and "PortofDischarge"

# Replace values in column "PortofDischarge" with the corresponding location names
merged_data["PortofDischarge"] = merged_data["PortofDischarge"].replace({
    "urn:jaif:id:loc:26LUSCHS": "Hafen Charleston",
    "urn:jaif:id:loc:26LUSSAV": "Bahnterminal Savannah",
    "urn:jaif:id:loc:26LMXATM": "Hafen Altamira"
})
# Assigning datatype "str" to "PortofDischarge"
merged_data["PortofDischarge"] = merged_data["PortofDischarge"].astype(str)
# Replace empty values and Whitespace in column PortofDischarge with NaN
merged_data["PortofDischarge"].replace("", np.nan, inplace=True)
merged_data["PortofDischarge"].replace(" ", np.nan, inplace=True)  

In [177]:
# Cell 8: Display the updated DataFrame
print(merged_data.head())

  ContainerNumber  ShipmentNumber    PoCreationDate     Material  \
0     HLXU8170197         4868820  19.02.2018 00:00  A2760101414   
1     HLXU8170197         4868820  19.02.2018 00:00  A2760101414   
2     HLXU8170197         4868820  19.02.2018 00:00  A2760101414   
3     HLXU8170197         4868820  19.02.2018 00:00  A2760101414   
4     HLXU8170197         4868820  19.02.2018 00:00  A2760101414   

   ShipmentQuantity Controller        EventName         EventTime  \
0              72.0        T15     Loading Ship  07.04.2018 13:58   
1              72.0        T15   Departure Ship  07.04.2018 19:05   
2              72.0        T15   Unloading Ship  21.04.2018 06:19   
3              72.0        T15  Departure Truck  21.04.2018 11:42   
4              72.0        T15    Arrival Truck  08.05.2018 18:12   

   EventMessageTime   VesselName PortofDischarge  Time2Arrival  \
0  08.04.2018 03:19  NYK ROMULUS             nan         125.0   
1  09.04.2018 16:29  NYK ROMULUS            

In [178]:
# Cell 9: Cleanup date columns - deal with corrupted data

# Deal with unreadable values in column date columns
# Check if "#" is part of the three date columns and remove rows with this sign
# Regular expression that checks for value "#"
regular_expression_pattern = "#"

# Check in each of the three columns and remove the rows with "#" characters
for column in ["PoCreationDate", "EventTime", "EventMessageTime"]:
    if column in merged_data.columns:
        contains_invalid_chars = merged_data[column].astype(str).str.contains(regular_expression_pattern)
        if contains_invalid_chars.any():
            print(f"Invalid characters found in column {column}. Remove affected lines.")
            merged_data = merged_data[~contains_invalid_chars]
        else:
            print(f"No invalid characters in column {column}")
    else:
        print(f"Column {column} does not exist in the DataFrame.")

print(f"Number of lines after removal:{len(merged_data)}")

No invalid characters in column PoCreationDate
No invalid characters in column EventTime
Invalid characters found in column EventMessageTime. Remove affected lines.
Number of lines after removal:35024


In [179]:
# Cell 10: Deal with missing values in "ShipmentQuantity"

# Deal with empty values of column ShipmentQuantity - Exploration reveals, all empty values can be linked to Material value "A2560107300"
# Calculate the mean of "ShipmentQuantity" for "A2560107300", excluding NaN values in the calculation
mean_shipment_quantity = merged_data.loc[merged_data["Material"] == "A2560107300", "ShipmentQuantity"].mean()

# Round the mean to the nearest integer if it is a valid number
mean_shipment_quantity = round(mean_shipment_quantity)

# Replace NaN values in "ShipmentQuantity" for "A2560107300" with the mean
merged_data.loc[(merged_data["Material"] == "A2560107300") & (merged_data["ShipmentQuantity"].isna()), "ShipmentQuantity"] = mean_shipment_quantity
print("Replaced NaN values in 'ShipmentQuantity' for 'A2560107300' with:", mean_shipment_quantity)

Replaced NaN values in 'ShipmentQuantity' for 'A2560107300' with: 35


In [180]:
# Cell 11: Additional data cleanup and type assignment
merged_data["PoCreationDate"] = pd.to_datetime(merged_data["PoCreationDate"], format="%d.%m.%Y %H:%M", errors='coerce')
merged_data["EventTime"] = pd.to_datetime(merged_data["EventTime"], format="%d.%m.%Y %H:%M", errors='coerce')
merged_data["EventMessageTime"] = pd.to_datetime(merged_data["EventMessageTime"], format="%d.%m.%Y %H:%M", errors='coerce')
merged_data["ContainerNumber"] = merged_data["ContainerNumber"].astype(str)
merged_data["VesselName"] = merged_data["VesselName"].astype(str)
merged_data["ShipmentNumber"] = merged_data["ShipmentNumber"].astype(int)
merged_data["ShipmentQuantity"] = merged_data["ShipmentQuantity"].astype(int)
merged_data["Material"] = merged_data["Material"].astype(str)
merged_data["Controller"] = merged_data["Controller"].astype(str)
merged_data["EventName"] = merged_data["EventName"].astype(str)
merged_data["Description"] = merged_data["Description"].astype(str)

In [181]:
# Cell 12: Deal with cryptic codes in column "EventLocation"

# Deal with value "urn:jaif:id:loc:25LUN498999044W013" whis is most likely a typo in column EventLocation. Replace with "Werk Tuscaloosa"
merged_data["Description"] = merged_data["Description"].replace("urn:jaif:id:loc:25LUN498999044W013", "Werk Tuscaloosa")
# Check the changes
print(merged_data["Description"])

0         Hafen Rotterdam
1         Hafen Rotterdam
2        Hafen Charleston
3        Hafen Charleston
4        Hafen Charleston
               ...       
35023    Hafen Charleston
35024    Hafen Charleston
35025    Hafen Charleston
35026     Werk Tuscaloosa
35027     Werk Tuscaloosa
Name: Description, Length: 35024, dtype: object


In [182]:
# Cell 13: Calculate values for column "Time2Arrival"

# Deal with empty values for Time2Arrival - calculate the number
# Sort the Dataframe based on "ShipmentNumber" and "EventTime" to ensure the events are in chronological order
merged_data.sort_values(by=["ShipmentNumber", "EventTime"], inplace=True)

# Calculate "Time2Arrival" as the number of days from the first event for each "ShipmentNumber" to the "Goods Receipt Dock" event
def calculate_time_to_arrival(df):
    # Calculate the time difference in days from the first event for each "ShipmentNumber"
    first_event_times = df.groupby("ShipmentNumber")["EventTime"].transform("max")
    df["Time2Arrival"] = -1 * (df["EventTime"] - first_event_times).dt.days
    return df

# Apply the function to the DataFrame
merged_data = calculate_time_to_arrival(merged_data)

# Ensure "Time2Arrival" is of type int
merged_data["Time2Arrival"] = merged_data["Time2Arrival"].astype(int)

In [183]:
# Cell 14: Replace German expression with English epressions 

# Replace German with English in "PortofDischarge" and "Description" columns
merged_data["PortofDischarge"] = merged_data["PortofDischarge"].str.replace("Hafen", "Port")
merged_data["Description"] = merged_data["Description"].str.replace("Hafen", "Port")
merged_data["PortofDischarge"] = merged_data["PortofDischarge"].str.replace("Werk", "Plant")
merged_data["Description"] = merged_data["Description"].str.replace("Werk", "Plant")

In [184]:
# Cell 15: Final data quality analysis and exporting the DataFrame to CSV

data_sample_count = merged_data.isnull().sum()
percent_missing = round(((data_sample_count.sum() / np.prod(merged_data.shape)) * 100), 2)
print("Count of missing values", "\n", data_sample_count)
print("Percent of missing data:", percent_missing)

merged_data.to_csv("exported_data_clean.csv", index=False, sep=";")

Count of missing values 
 ContainerNumber     0
ShipmentNumber      0
PoCreationDate      0
Material            0
ShipmentQuantity    0
Controller          0
EventName           0
EventTime           0
EventMessageTime    0
VesselName          0
PortofDischarge     0
Time2Arrival        0
Description         0
dtype: int64
Percent of missing data: 0.0
